In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import util as u
import json
import pickle


BASE_PATH = "../"

In [2]:
all_df = pd.read_csv(Path(BASE_PATH,"metadata.csv"))

In [3]:
def is_manually_checked(row):
    #check based on the audio click files
    if Path(BASE_PATH,row["midi_performance"][:-4]+ "_click.wav").exists():
        return False
    else:
        return True
    
def key_number_from_number_of_sharps(nos):
    if nos >= 0:
        return int((nos * 7)%12)
    else:
        return int((np.abs(nos)*5)%12)
    
type(key_number_from_number_of_sharps(-3))

int

In [4]:
def get_beats_from_txt(ann_path):
    ann_df = pd.read_csv(Path(BASE_PATH,ann_path),header=None, names=["time","time2","type"],sep='\t')
    return ann_df["time"].tolist()

def get_downbeats_from_txt(ann_path):
    ann_df = pd.read_csv(Path(BASE_PATH,ann_path),header=None, names=["time","time2","type"],sep='\t')
    downbeats = [a["time"] for i,a in ann_df.iterrows() if a["type"].split(",")[0] == "db"]
    return downbeats

def get_beats_db_dict_from_txt(ann_path):
    ann_df = pd.read_csv(Path(BASE_PATH,ann_path),header=None, names=["time","time2","type"],sep='\t')
    out_dict = {str(a["time"]): a["type"].split(",")[0] for i,a in ann_df.iterrows()}
    return out_dict

def get_key_from_txt(ann_path):
    ann_df = pd.read_csv(Path(BASE_PATH,ann_path),header=None, names=["time","time2","type"],sep='\t')
    keys = {}
    for i, r in ann_df.iterrows():
        if len(r["type"].split(","))==3:
            number_of_sharps = int(r["type"].split(",")[2])
            key_number = key_number_from_number_of_sharps(number_of_sharps)
            keys[str(r["time"])] = [key_number, number_of_sharps]
    return keys

def get_ts_from_txt(ann_path):
    ann_df = pd.read_csv(Path(BASE_PATH,ann_path),header=None, names=["time","time2","type"],sep='\t')
    tss = {}
    for i, r in ann_df.iterrows():
        if len(r["type"].split(","))>1 and r["type"].split(",")[1]!="":
            ts = r["type"].split(",")[1]
            beat_n = int(u.ts2n_of_beats(ts))
            tss[str(r["time"])] = [ts, beat_n]
    return tss

def midi_and_score_aligned(row):
    return len(get_beats_from_txt(row["performance_annotations"])) == len(get_beats_from_txt(row["midi_score_annotations"]))


In [5]:
asap_ann = {}
print("Creating json")

counter = 0
for i,row in all_df.iterrows():
    asap_ann[row["midi_performance"]] = {
        "performance_beats" : get_beats_from_txt(row["performance_annotations"]),
        "performance_downbeats" : get_downbeats_from_txt(row["performance_annotations"]),
        "performance_beats_type" : get_beats_db_dict_from_txt(row["performance_annotations"]),
        "perf_time_signatures" : get_ts_from_txt(row["performance_annotations"]),
        "perf_key_signatures" : get_key_from_txt(row["performance_annotations"]),
        "midi_score_beats" : get_beats_from_txt(row["midi_score_annotations"]),
        "midi_score_downbeats" : get_downbeats_from_txt(row["midi_score_annotations"]),
        "midi_score_beats_type" : get_beats_db_dict_from_txt(row["midi_score_annotations"]),
        "midi_score_time_signatures" : get_ts_from_txt(row["midi_score_annotations"]),
        "midi_score_key_signatures" : get_key_from_txt(row["midi_score_annotations"]),
        "downbeats_score_map" : u.same_number_of_measures_with_repetitions(row["xml_score"], str(Path(BASE_PATH,row["midi_score_annotations"])), base_path=BASE_PATH),
        "score_and_performance_aligned" : midi_and_score_aligned(row) ,
#         "manually_checked" : is_manually_checked(row)
    }
    counter +=1
    print("#",end="")
    

Creating json
##########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

In [6]:
with open(Path('../asap_annotations.json'), 'w') as outfile:
    json.dump(asap_ann, outfile)